# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
cities = pd.read_csv("Output/CityData.csv", encoding="utf-8")
cities.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Country,Windspeed,Date
0,Oromocto,45.84,-66.48,52.00,71,90,CA,5.82,1603471314
1,Saint george,32.38,-64.68,80.01,83,75,BM,19.46,1603471314
2,Qaanaaq,77.48,-69.36,20.05,94,100,GL,9.60,1603471314
3,Arman,59.70,150.17,36.14,85,100,RU,10.25,1603471314
4,Bredasdorp,-34.53,20.04,72.00,77,41,ZA,4.00,1603471315


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Lat", "Lng"]]

In [26]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
city_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Cloudiness"] == 0), :]
city_df = city_df.dropna(how='any')
city_df.reset_index(inplace=True)
del city_df['index']
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Country,Windspeed,Date
0,Saint-philippe,-21.36,55.77,73.40,78,0,RE,14.99,1603471384
1,Chongwe,-15.33,28.68,75.92,29,0,ZM,5.53,1603471516
2,Viedma,-40.81,-63.00,77.00,52,0,AR,5.01,1603471516
3,Mizdah,31.45,12.98,71.13,29,0,LY,9.24,1603471517
4,Saint-philippe,-21.36,55.77,73.40,78,0,RE,14.99,1603471384


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
hotel = []

for i in range(len(city_df)):
    lat = city_df.loc[i]['Lat']
    lng = city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotel.append(jsn['results'][0]['name'])
    except:
        hotel.append("")
city_df["Hotel Name"] = hotel
city_df = city_df.dropna(how='any')
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Country,Windspeed,Date,Hotel Name
0,Saint-philippe,-21.36,55.77,73.40,78,0,RE,14.99,1603471384,Saint-Philippe
1,Chongwe,-15.33,28.68,75.92,29,0,ZM,5.53,1603471516,Chongwe
2,Viedma,-40.81,-63.00,77.00,52,0,AR,5.01,1603471516,Viedma
3,Mizdah,31.45,12.98,71.13,29,0,LY,9.24,1603471517,Mizdah
4,Saint-philippe,-21.36,55.77,73.40,78,0,RE,14.99,1603471384,Saint-Philippe


In [29]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in city_df.iterrows()]
locations = city_df[["Lat", "Lng"]]

In [30]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))